In [57]:
import pandas as pd
import numpy as np 
import time
from datetime import datetime, timedelta

In [58]:
# read raw data (downloaded from http://web.mta.info/developers/turnstile.html) to dataframe 

df_march_3 = pd.read_csv('3_3_2018.csv')
df_march_10 = pd.read_csv('3_10_2018.csv')
df_march_17 = pd.read_csv('3_17_2018.csv')
df_march_24 = pd.read_csv('3_24_2018.csv')
df_march_31 = pd.read_csv('3_31_2018.csv')
df_april_7 = pd.read_csv('4_7_2018.csv')
df_april_14 = pd.read_csv('4_14_2018.csv')
df_april_21 = pd.read_csv('4_21_2018.csv')
df_april_28 = pd.read_csv('4_28_2018.csv')
df_may_5 = pd.read_csv('5_5_2018.csv')
df_may_12 = pd.read_csv('5_12_2018.csv')
df_may_19 = pd.read_csv('5_19_2018.csv')
df_may_26 = pd.read_csv('5_26_2018.csv')
df_june_2 = pd.read_csv('6_2_2018.csv')
df_june_9 = pd.read_csv('6_9_2018.csv')
df_june_16 = pd.read_csv('6_16_2018.csv')

In [59]:
# combine individual dataframes into a single merged dataframes 

frames = [df_march_3, df_march_10, df_march_17, df_march_24, df_march_31, df_april_7, df_april_14, df_april_21, df_april_28, df_may_5, df_may_12, df_may_19, df_may_26, df_june_2, df_june_9, df_june_16]
merge = pd.concat(frames)

In [60]:
# remove white space from column names (specifically EXITS) and create new columns in merge dataframe

new_names = [name.strip() for name in merge.columns]
merge.columns = new_names

In [61]:
# Remove "RECOVR AUD" entries (in DESC column), which refers to a missed audit that was recovered

merge = merge[merge['DESC'] != 'RECOVR AUD']

In [62]:
# convert DATE column to datetime
# create 'month' and 'day_of_week' columns

merge['DATE'] = pd.to_datetime(merge['DATE'])
merge['month'] = merge['DATE'].dt.month
merge['day_of_week'] = merge['DATE'].dt.weekday_name

In [63]:
# function to split data into correct day buckets (assigns midnight bucket to the previous day)
def subtractOneDay(day):
    
    """
    In this dataset, a 'TIME' of 00:00:00 represents the 4 last hours of the previous day
    (20:00:00 - 00:00:00). However, the corresponding day in the 'day_of_week' column of
    the same row is the day associated with 00:00:00. For example, 00:00:00 Saturday really
    represents data collected on Friday from 20:00:00 to 00:00:00. This function assigns the 
    day_of_week associated with any 'TIME' equal to 00:00:00 to the previous day. 
    """
        
    if day == 'Saturday': 
        new_day = 'Friday'
    elif day == 'Friday': 
        new_day = 'Thursday'
    elif day == 'Thursday': 
        new_day = 'Wednesday'
    elif day == 'Wednesday': 
        new_day = 'Tuesday'
    elif day == 'Tuesday': 
        new_day = 'Monday'
    elif day == 'Monday': 
        new_day = 'Sunday'
    elif day == 'Sunday': 
        new_day = 'Saturday'
    return new_day

In [64]:
# reset index and drop old index to avoid getting a Value Error when running SubtractOneDay function
merge = merge.reset_index(drop=True)

In [65]:
# applies subtractOneDay() function to 'TIME' column and assigns data to new column
merge['new_day'] = merge.day_of_week[merge['TIME'] == '00:00:00'].apply(subtractOneDay)
# fills in null values in new column with corresonding value from 'day_of_week'
merge['new_day'] = merge['new_day'].fillna(merge['day_of_week'])

In [66]:
# function to group time intervals from 'TIME' column into categories in new column
def convertTimeToCategory(time):
    
    """
    This function creates a new column that groups time intervals into categories:
    
    00:00 < late night <= 4:00
    4:00 < early risers <= 8:00
    8:00 < morning <= 12:00
    12:00 < afternoon <= 16:00
    16:00 < evening  <= 20:00
    20:00 < night <= 00:00
    """
    
    hour = time.hour
    if hour > 20 or hour == 0:
        category = 'Night'
    elif hour > 16:
        category = 'Evening'
    elif hour > 12:
        category = 'Afternoon'
    elif hour > 8:
        category = 'Morning'
    elif hour > 4:
        category = 'Early Risers'
    elif hour > 0:
        category = 'Late Night'
    
    return category

In [67]:
# apply convertTimeToCategory() function

merge['TIME_2'] = pd.to_datetime(merge['TIME']).dt.round('H')
merge['Category'] = merge['TIME_2'].apply(convertTimeToCategory)
merge.drop(columns=['TIME_2'], inplace = True)
# merge['TIME'] = pd.to_timedelta(merge['TIME'], unit='H')
# merge['Category'] = merge['TIME'].apply(convertTimeToCategory)

In [68]:
# compute values for number of people entering and exiting a turnstile for a given time stamp

merge['Ent'] = merge['ENTRIES'].diff()
merge['Ext'] = merge['EXITS'].diff()

In [69]:
# remove NaN values in row 0

merge = merge.dropna(axis=0)

In [70]:
# remove negative values for Ent or Ext

merge = merge[merge['Ent'] >= 0]
merge = merge[merge['Ext'] >= 0]

In [71]:
# Use Dan's magic_number function to remove outliers
# See 'getCutoff.ipynb' for source for these numbers

merge = merge[merge['Ent'] <= 312943]
merge = merge[merge['Ext'] <= 143941]

In [72]:
# Sum Ent and Ext to compute total number of people moving through a single turnstile in a given time frame

merge['traffic'] = merge['Ent'] + merge['Ext']

In [73]:
# move cleaned data to a csv file
merge.to_csv('clean_merged_mta_data.csv', index = False)